# Documentation

```https://docs.ultralytics.com/```

# Import Dependencies

In [ ]:
#import shutil 
#shutil.rmtree("/kaggle/working/runs")

In [1]:
!pip install ultralytics
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 713.3 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.4/799.4 kB 6.3 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
pip install -U "ray[data,train,tune,serve]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 26.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 MB 25.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.2/561.2 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.9.0
    Uninstalling ray-2.9.0:
      Successfully uninstalled ray-2.9.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import shutil
import ultralytics
import pandas as pd
from PIL import Image
from ultralytics import YOLO
from IPython.display import Image
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler

2024-07-07 22:07:47,381	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-07 22:07:47,607	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Check Dependencies

In [4]:
torch.__version__

'2.1.2'

In [5]:
# Torch cuda version must same as nvidia cuda version
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


In [6]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0)

In [7]:
ultralytics.checks()

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.6/8062.4 GB disk)


# Dataset Location

In [8]:
dataset1 = "/kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset"

# DATA .YAML

In [9]:
data1 = "/kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/data.yaml"

# Pre-Trained Model Load

```yolov8.pt is a pre-trained YOLOv8 model```

In [10]:
ptmodel1 = YOLO('yolov8n.pt')
ptmodel1

100%|██████████| 6.25M/6.25M [00:00<00:00, 75.4MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

# Train Model

In [11]:
# Load a model
#model = YOLO("/kaggle/working/runs/detect/train2/weights/last.pt")  # load a partially trained model

# Resume training
#results = model.train(resume=True)

In [12]:
# wandp API e96c755950defb66d26359bf9da9eb53c98b5196
train1 = ptmodel1.train(data=data1, lr0=0.001, epochs=50, patience=30, batch=10)

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/data.yaml, epochs=50, time=None, patience=30, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

100%|██████████| 755k/755k [00:00<00:00, 13.6MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/train/labels... 352 images, 0 backgrounds, 0 corrupt: 100%|██████████| 352/352 [00:01<00:00, 335.54it/s]

train: WARNING ⚠️ Cache directory /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/train is not writeable, cache not saved.


...

      50/50      1.45G     0.2776     0.2587      0.783          2        640: 100%|██████████| 36/36 [00:04<00:00,  7.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:00<00:00,  7.88it/s]

                   all        103        103      0.945      0.989      0.991      0.957



50 epochs completed in 0.095 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.88it/s]


                   all        103        103       0.95      0.988      0.991      0.956
                 awake         59         59      0.945          1       0.99      0.948
                drowsy         44         44      0.955      0.975      0.991      0.963
Speed: 0.1ms preprocess, 3.1ms inference, 0.0ms loss, 13.2ms postprocess per image
Results saved to runs/detect/train


# Validate Model

In [13]:
# Load a model
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

# Customize validation settings
validation_test_results1 = model.val(data=data1, imgsz=640, batch=10, conf=0.25, iou=0.6, split="test")

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/labels... 71 images, 0 backgrounds, 0 corrupt: 100%|██████████| 71/71 [00:00<00:00, 335.20it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test is not writeable, cache not saved.



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.19it/s]


                   all         71         71      0.949      0.959      0.948      0.914
                 awake         40         40       0.96       0.95      0.962       0.92
                drowsy         31         31      0.938      0.968      0.934      0.908
Speed: 0.2ms preprocess, 10.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val


# Validate Function

In [22]:
import pandas as pd
import numpy as np

def calc_conf_matrix(conf_matrix):
    # Make a dataframe for the results
    results_df = pd.DataFrame(columns=["Class", "TP", "FP", "FN", "TN", "Precision", "Recall", "F1-Score", "Accuracy"])

    # Iterate through classes
    for class_index in range(len(conf_matrix)):

        # Confusion Matrix Data
        tp = conf_matrix[class_index][class_index]
        fp = sum(conf_matrix[:, class_index]) - tp
        fn = sum(conf_matrix[class_index, :]) - tp
        tn = np.sum(conf_matrix) - (tp + fp + fn)

        # Evaluation
        precision = tp / (tp + fp) if (tp + fp) > 0 else float('nan')
        recall = tp / (tp + fn) if (tp + fn) > 0 else float('nan')
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else float('nan')
        accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else float('nan')

        # Add to df
        row = pd.DataFrame([{
            "Class": class_index,
            "TP": tp,
            "FP": fp,
            "FN": fn,
            "TN": tn,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1_score,
            "Accuracy": accuracy
        }])
        results_df = pd.concat([results_df, row], ignore_index=True)
    
    # Calculate the average accuracy
    avg_accuracy = results_df["Accuracy"].mean()

    # Add the average accuracy row
    avg_accuracy_row = pd.DataFrame([{
        "Class": "Average",
        "TP": None,
        "FP": None,
        "FN": None,
        "TN": None,
        "Precision": None,
        "Recall": None,
        "F1-Score": None,
        "Accuracy": avg_accuracy
    }])

    results_df = pd.concat([results_df, avg_accuracy_row], ignore_index=True)

    return results_df

# Training Confusion Matrix

In [19]:
train_conf_matrix1 = train1.confusion_matrix.matrix
train_conf_matrix1

array([[         58,           1,           3],
       [          1,          43,           1],
       [          0,           0,           0]])

# Calculate Train Confusion Matrix

In [23]:
train_conf_matrix_result1 = calc_conf_matrix(train_conf_matrix1)
train_conf_matrix_result1

,Class,TP,FP,FN,TN,Precision,Recall,F1-Score,Accuracy
0,0,58.0,1.0,4.0,44.0,0.983051,0.935484,0.958678,0.953271
1,1,43.0,1.0,2.0,61.0,0.977273,0.955556,0.966292,0.971963
2,2,0.0,4.0,0.0,103.0,0.000000,NaN,NaN,0.962617
3,Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.962617


# Testing Confusion Matrix

In [24]:
test_conf_matrix1 = validation_test_results1.confusion_matrix.matrix
test_conf_matrix1

array([[         38,           1,           2],
       [          2,          30,           0],
       [          0,           0,           0]])

# Calculate Test Confusion Matrix

In [25]:
test_conf_matrix_result1 = calc_conf_matrix(test_conf_matrix1)
test_conf_matrix_result1

,Class,TP,FP,FN,TN,Precision,Recall,F1-Score,Accuracy
0,0,38.0,2.0,3.0,30.0,0.950000,0.926829,0.938272,0.931507
1,1,30.0,1.0,2.0,40.0,0.967742,0.937500,0.952381,0.958904
2,2,0.0,2.0,0.0,71.0,0.000000,NaN,NaN,0.972603
3,Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.954338


# Accuracy Graphs

In [26]:
pip install pandas openpyxl

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [27]:
# Train Confusion Matrix Results Excel

conf_matrix_results = [train_conf_matrix_result1, test_conf_matrix_result1]

# Reorder columns for better readability
results_df = pd.DataFrame(columns=["Class", "TP", "FP", "FN", "TN", "Precision", "Recall", "F1-Score", "Accuracy"])


# Menyiapkan nama file dan path di Google Drive
file_name = 'train&valid_conf_matrix_results_ori.xlsx'
file_path = '/kaggle/working/runs/' + file_name  # Sesuaikan dengan path di Google Drive Anda

# Membuat DataFrame dari conf_matrix_results
df_list = []

for idx, conf_matrix_result in enumerate(conf_matrix_results):
    df = pd.DataFrame(conf_matrix_result)
    df['Iteration'] = idx + 1  # Menambahkan kolom untuk nomor iterasi
    df_list.append(df)

# Menggabungkan semua DataFrame menjadi satu
final_df = pd.concat(df_list, ignore_index=True)

# Menyimpan DataFrame ke dalam file Excel di Google Drive
final_df.to_excel(file_path, index=False)

print(f"Excel report has been created and saved {file_path}")


Excel report has been created and saved /kaggle/working/runs/train&valid_conf_matrix_results_ori.xlsx


# Predict

In [28]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [33]:
!yolo task=detect mode=predict model="/kaggle/working/runs/detect/train/weights/best.pt" conf=0.50 source="/kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images"

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

image 1/71 /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images/GOPR0492_MP4-100_jpg.rf.3151be21e0045f64fb6800cf58e3983d.jpg: 384x640 1 drowsy, 73.3ms
image 2/71 /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images/GOPR0492_MP4-109_jpg.rf.2c20d7b81d458d9a8bbbbb717d6ea779.jpg: 384x640 1 awake, 7.3ms
image 3/71 /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images/GOPR0492_MP4-117_jpg.rf.93aa7b9daec5cc05c1b082625bb00846.jpg: 384x640 1 drowsy, 7.1ms
image 4/71 /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images/GOPR0492_MP4-121_jpg.rf.6cfcbce35d27385803184b436e72f3e8.jpg: 384x640 1 awake, 6.9ms
image 5/71 /kaggle/input/drowsy-detection-dataset/drowsy-detection-dataset/test/images/GOPR0492_MP4-126_jpg.rf.c869025e590c717a9087896123046422.jpg:

In [34]:
!yolo task=detect mode=predict model="/kaggle/working/runs/detect/train/weights/best.pt" conf=0.50 source="/kaggle/input/test-video/test_video.mp4"

Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (frame 1/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 73.2ms
video 1/1 (frame 2/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.6ms
video 1/1 (frame 3/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.8ms
video 1/1 (frame 4/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 7.0ms
video 1/1 (frame 5/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.1ms
video 1/1 (frame 6/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.3ms
video 1/1 (frame 7/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 7.8ms
video 1/1 (frame 8/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.1ms
video 1/1 (frame 9/496) /kaggle/input/test-video/test_video.mp4: 384x640 1 awake, 6.3ms
video 1/1 (frame 10/496) /kaggle/input/test-vi

In [35]:
!zip -r ori_output.zip /kaggle/working/

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/runs/ (stored 0%)
updating: kaggle/working/runs/detect/ (stored 0%)
updating: kaggle/working/runs/detect/val/ (stored 0%)
updating: kaggle/working/runs/detect/val/PR_curve.png (deflated 21%)
updating: kaggle/working/runs/detect/val/confusion_matrix.png (deflated 36%)
updating: kaggle/working/runs/detect/val/val_batch2_pred.jpg (deflated 12%)
updating: kaggle/working/runs/detect/val/F1_curve.png (deflated 18%)
updating: kaggle/working/runs/detect/val/val_batch1_labels.jpg (deflated 12%)
updating: kaggle/working/runs/detect/val/R_curve.png (deflated 18%)
updating: kaggle/working/runs/detect/val/val_batch0_labels.jpg (deflated 12%)
updating: kaggle/working/runs/detect/val/val_batch2_labels.jpg (deflated 12%)
updating: kaggle/working/runs/detect/val/val_batch0_pred.jpg (deflated 11%)
updating: kaggle/working/runs/detect/val/confusion_matrix_normalized.png (deflated 34%)
updating: kaggle/working/runs/detect/val/P_curve.png (defl

In [36]:
from IPython.display import FileLink
FileLink(r'ori_output.zip')

/kaggle/working/ori_output.zip